Funkcja ukrywająca wiadomość przy użyciu znaków o zerowej szerokości

Parametry:
- plain_text (str): Tekst, do którego chcemy dodać ukrytą wiadomość.
- secret_message (str): Wiadomość do ukrycia.

Zwraca:
- encoded_text (str): Tekst oryginalny z dodaną zaszyfrowaną wiadomością.

In [1]:
def encrypt(plain_text, secret_message):

    # Kodowanie ukrytej wiadomości na postać binarną
    binary_message = ''.join(format(ord(char), '08b') for char in secret_message)
    zero_width_encoding = binary_message.replace('0', '\u200B').replace('1', '\u200C')

    # Łączenie tekstu oryginalnego z zakodowaną wiadomością
    encoded_text = plain_text + '\u200D' + zero_width_encoding + '\u200D'
    return encoded_text

Funkcja odszyfrowuje ukrytą wiadomość z zakodowanego tekstu.

Parametry:
- encoded_text (str): Tekst z ukrytą wiadomością.

Zwraca:
- plain_text (str): Tekst oryginalny bez ukrytej wiadomości.
- secret_message (str): Ukryta wiadomość.

In [2]:
def decrypt(encoded_text):
    # Sprawdzanie, czy w tekście znajduje się ukryta wiadomość
    if '\u200D' not in encoded_text:
        return "Brak ukrytej wiadomości.", encoded_text

    # Rozdzielanie tekstu oryginalnego i zakodowanej wiadomości
    plain_text, encoded_message = encoded_text.split('\u200D', 1)
    encoded_message = encoded_message.strip('\u200D')

    # Konwersja znaków o zerowej szerokości z powrotem do postaci binarnej
    binary_message = encoded_message.replace('\u200B', '0').replace('\u200C', '1')

    # Konwersja binarnej postaci na znaki, aby uzyskać ukrytą wiadomość
    secret_message = ''.join(chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8))

    return plain_text, secret_message

Przykład użycia:

In [3]:
plain_text = "Hello, jak się masz?"
secret_message = "Ukryta123"

# Szyfrowanie
encoded_text = encrypt(plain_text, secret_message)
print("Zakodowany tekst:", encoded_text)

# Odszyfrowanie
original_text, decoded_message = decrypt(encoded_text)
print("Tekst oryginalny:", original_text)
print("Odszyfrowana wiadomość:", decoded_message)

Zakodowany tekst: Hello, jak się masz?‍​‌​‌​‌​‌​‌‌​‌​‌‌​‌‌‌​​‌​​‌‌‌‌​​‌​‌‌‌​‌​​​‌‌​​​​‌​​‌‌​​​‌​​‌‌​​‌​​​‌‌​​‌‌‍
Tekst oryginalny: Hello, jak się masz?
Odszyfrowana wiadomość: Ukryta123


**Druga implementacja**

from faker import Faker

# Initialize Faker
fake = Faker()


# Function to determine bit based on first letter
def letter_to_bit(letter):
    vowels = "AEIOU"
    return '0' if letter.upper() in vowels else '1'


# Function to encode a single byte as a department
def generate_department(byte):
    department = []
    bits = f"{byte:08b}"  # Convert byte to 8-bit binary string

    for i in range(0, 8, 2):  # Each person contributes 2 bits
        # Generate random names until we get one with the right first-letter bits
        while True:
            name = fake.first_name()
            surname = fake.last_name()
            name_bit = letter_to_bit(name[0])
            surname_bit = letter_to_bit(surname[0])

            if name_bit == bits[i] and surname_bit == bits[i + 1]:
                department.append((name, surname))
                break

    return department


# Function to encode a message into departments
def encode_message(message):
    byte_array = message.encode('utf-8')  # Encode message to bytes
    departments = []

    for byte in byte_array:
        department = generate_department(byte)
        departments.append(department)

    return departments


# Function to decode the message from departments
def decode_message(departments):
    bytes_list = []

    for department in departments:
        bits = ""
        for name, surname in department:
            # Extract bits from the first letters of name and surname
            name_bit = letter_to_bit(name[0])
            surname_bit = letter_to_bit(surname[0])
            bits += name_bit + surname_bit

        # Convert 8 bits to a byte and append to the list
        byte = int(bits, 2)
        bytes_list.append(byte)

    # Convert the list of bytes back to the original string
    decoded_message = bytes(bytes_list).decode('utf-8')
    return decoded_message


if __name__ == "__main__":
    secret_message = "Hello"
    encoded_departments = encode_message(secret_message)

    print("Encoded Departments:")
    for i, department in enumerate(encoded_departments):
        print(f"Department {i + 1}:")
        for name, surname in department:
            print(f"  - {name} {surname}")

    # Decode the message
    decoded_message = decode_message(encoded_departments)
    print("\nDecoded Message:", decoded_message)


**Homoglify**


Program wykorzystuje steganografię tekstową do ukrywania wiadomości w zwykłym tekście poprzez zamianę wybranych liter na ich homoglifowe odpowiedniki z alfabetu cyrylickiego i greckiego. Wiadomość jest konwertowana na ciąg binarny, a następnie kodowana w tekście, zastępując litery zgodnie z wartością bitu (1 → homoglif, 0 → oryginalny znak). Jeśli tekst bazowy jest za krótki, do jego rozszerzenia używany jest model GPT-2, który generuje dodatkową treść. Program umożliwia zarówno zakodowanie, jak i późniejsze odczytanie ukrytej wiadomości, analizując występowanie homoglifów w tekście zakodowanym.

importy:

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

Ładowanie modelu GPT-2 i tokenizera

In [ ]:
model_name = "gpt2" 
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Homoglify:

In [ ]:
HOMOGLYPHS = {
    # Cyrylica (małe litery)
    'a': 'а',  # Cyrylica 'a'
    'e': 'е',  # Cyrylica 'e'
    'o': 'о',  # Cyrylica 'o'
    'p': 'р',  # Cyrylica 'p'
    'c': 'с',  # Cyrylica 'c'
    'x': 'х',  # Cyrylica 'x'
    'y': 'у',  # Cyrylica 'y'

    # Grecki (wielkie litery)
    'A': 'Α',  # Grecka 'A'
    'B': 'Β',  # Grecka 'B'
    'E': 'Ε',  # Grecka 'E'
    'H': 'Η',  # Grecka 'H'
    'I': 'Ι',  # Grecka 'I'
    'K': 'Κ',  # Grecka 'K'
    'M': 'Μ',  # Grecka 'M'
    'N': 'Ν',  # Grecka 'N'
    'O': 'Ο',  # Grecka 'O'
    'P': 'Ρ',  # Grecka 'P'
    'T': 'Τ',  # Grecka 'T'
    'X': 'Χ',  # Grecka 'X'
    'Y': 'Υ',  # Grecka 'Y'

    # Cyrylica (wielkie litery) 
    'C': 'С',  # Cyrylica 'C'
}

# Generowanie odwrotnej mapy
REVERSE_HOMOGLYPHS = {v: k for k, v in HOMOGLYPHS.items()}

Funkcja generująca sensowną kontynuację tekstu:

In [ ]:
def generate_continuation_with_gpt2(previous_text, max_length=150):
    """
    Generuje sensowną kontynuację tekstu za pomocą modelu GPT-2.
    """
    # Tokenizacja poprzedniego tekstu
    input_ids = tokenizer.encode(previous_text, return_tensors='pt')

    # Tworzenie maski uwagi (wszystkie tokeny mają wartość 1, bo nie używamy paddingu)
    attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

    # Generowanie kontynuacji z uwzględnieniem maski uwagi
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7, do_sample=True, pad_token_id=50256)

    # Dekodowanie wyników
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

Dekodowanie wiadomości:

In [ ]:
def decode_message(encoded_text):
    """
    Dekoduje wiadomość z zakodowanego tekstu.
    """
    binary_message = []
    
    for char in encoded_text:
        if char in REVERSE_HOMOGLYPHS:
            binary_message.append('1')
        elif char in HOMOGLYPHS:
            binary_message.append('0')
    
    # Debug: Wyświetl binarną wiadomość
    # print(f"Odczytana wiadomość binarna: {''.join(binary_message)}")
    
    # Grupujemy po 8 bitów (jeden znak w ASCII) i konwertujemy na tekst
    message_chars = [
        chr(int(''.join(binary_message[i:i+8]), 2))
        for i in range(0, len(binary_message), 8)
    ]
    return ''.join(message_chars)

Wywołanie programu:

In [ ]:
original_text = "Once upon a time, there was a brave hero named Alex."
hidden_message = "Ela z mlotkiem w Dubaju" # mniej więcej maksymalna długośc wiadomości
# hidden_message = "Hi" 

try:
    # Kodowanie
    encoded_text = encode_message(original_text, hidden_message)
    print("Zakodowany tekst:", encoded_text)

    # Dekodowanie
    decoded_message = decode_message(encoded_text)
    print("Odkodowana wiadomość:", decoded_message)
except ValueError as e:
    print(f"Błąd: {e}")